# Refinitiv Data Library for Python

## Delivery layer - Bulk - Client File Store - Tick History
This notebook demonstrates how to browse the Client File Store (CFS) of the Refinitiv Data Platform (RDP) using the Refintiv Data Library. The delivery.bulk.cfs module used in this example gives you an fine control on CFS operations. Unless you have a specific need, it is recommended to use the delivery.bulk.PackageManager instead. The Package manager provides you with a higher level API that manages for you the 3 types of activities generally invoved with bulk files (downloading files, populating databases, querying databases).

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
import refinitiv.data.delivery.cfs as cfs

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
rd.open_session('platform.rdp')

<refinitiv.data.session.platform.Definition object at 0x7fa5ccc6dd60 {session_name='rdp'}>

## Retrieve Tick History data

### Browse Tick History buckets

In [4]:
response = cfs.buckets.Definition('history').get_data()

buckets = response.data.buckets
for bucket in buckets:
    print("Bucket: ", bucket.name)
    print("\tFile sets: ")
    for file_set in bucket:
        print("\t\t{}\tID:{}".format(file_set.name, file_set.id))
        print("\t\t\tFiles: ")
        for file in file_set:
            print("\t\t\t\t{}\tSize (in Bytes):{}".format(file.filename, file.file_size_in_bytes))

Bucket:  TICKHISTORY_VBD_NO_EMBARGO
	File sets: 
		ICX_normalised_2012-11-24_UNLIMITED	ID:4000-0037-263d473f-920d-7d0b0396d012
			Files: 
				ICX-2012-11-24-NORMALIZEDMP-Report-1-of-1.csv.gz	Size (in Bytes):1113
				ICX-2012-11-24-NORMALIZEDMP-Data-1-of-1.csv.gz	Size (in Bytes):449
		LSE_normalised_2014-09-24_UNLIMITED	ID:4000-007a-a23e2dbb-8e9d-1f576e3876f4
			Files: 
				LSE-2014-09-24-NORMALIZEDMP-Report-1-of-1.csv.gz	Size (in Bytes):262657
				LSE-2014-09-24-NORMALIZEDMP-Data-1-of-1.csv.gz	Size (in Bytes):195705388
		KSC_rawLL2_2005-12-23_UNLIMITED	ID:4000-0087-4f771645-873b-6e1032de6468
			Files: 
				KSC-2005-12-23-LEGACYLEVEL2-Data-1-of-1.csv.gz	Size (in Bytes):91072714
				KSC-2005-12-23-LEGACYLEVEL2-Report-1-of-1.csv.gz	Size (in Bytes):54023
		MLT_ref-data-sedol-gics_2020-12-13_UNLIMITED	ID:4000-00ae-9f577cad-a77f-ecd6d188d6af
			Files: 
				MLT-2020-12-13-REF-Data-SEDOL-GICS-1-of-1.csv.gz	Size (in Bytes):1412
		DIX_rawLL2_2006-08-26_UNLIMITED	ID:4000-00c8-0533d86d-9713-2bdc4e

### Get file set details

In [5]:
fileset_id = '4000-0037-263d473f-920d-7d0b0396d012'
response = cfs.files.Definition(fileset_id).get_data()
response.data.df

,modified,fileSizeInBytes,fileType,md5,description,filename,filesetId,created,storageLocation,id,href
0,2019-12-12T22:03:12Z,1113,File,533d222b9033e1ea254c4beb63a50e30,Merged data file,ICX-2012-11-24-NORMALIZEDMP-Report-1-of-1.csv.gz,4000-0037-263d473f-920d-7d0b0396d012,2019-12-12T22:03:12Z,{'url': 'https://s3-us-west-1.amazonaws.com/ti...,438d-f6af-263f2c31-abfd-7dacc3c1c7f3,https://api.refinitiv.com/file-store/v1/files/...
1,2019-12-12T22:03:12Z,449,File,c66c3b1749dc885e222d3953e522e9b8,Merged data file,ICX-2012-11-24-NORMALIZEDMP-Data-1-of-1.csv.gz,4000-0037-263d473f-920d-7d0b0396d012,2019-12-12T22:03:12Z,{'url': 'https://s3-us-west-1.amazonaws.com/ti...,4a93-8ece-24d9988f-8409-7d06502a2dbc,https://api.refinitiv.com/file-store/v1/files/...


### Download files
For the sake of the example, only files under 500KB are downloaded

In [6]:
for file in response.data.files:
    print('Downloading "{}"...'.format(file.filename))
    if file.file_size_in_bytes > 500000:
        print('\t...File skipped.  "{}" is too big for this example. Size (Bytes): {}'.format(file.filename, file.file_size_in_bytes))
    else:
        file_downloader = cfs.file_downloader.Definition(file).retrieve()
        file_downloader.download()
        print('\t...Done')
        print('Extracting "{}"...'.format(file.filename))
        file_downloader.extract()
        print('\t...Done')

	...Done
Extracting "ICX-2012-11-24-NORMALIZEDMP-Report-1-of-1.csv.gz"...
	...Done
	...Done
Extracting "ICX-2012-11-24-NORMALIZEDMP-Data-1-of-1.csv.gz"...
	...Done


## Close the session

In [7]:
rd.close_session()